In [14]:
from datagen import *
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation
import json
from IPython.display import HTML


In [8]:
scene = generate_scene_2gals()

{'nbStars': 481, 'radius': 1.3364060066003773, 'Mass': 1.5004314904783438, 'zOffsetMax': 0.0011181212556133358, 'gravityCst': 1.0, 'distribution': 'hernquist', 'offset': [-0.37762432756890996, -0.5610089398651985, -0.4634166057718323], 'initial_vel': [-0.05341546724788393, 0.020162675731639684, 0.003208541599431264], 'clockwise': 1, 'angle': [0.16515403699799427, 3.3357729781165895, -2.0665159123239922]}
{'nbStars': 171, 'radius': 1.3946615766020227, 'Mass': 1.4831877817307035, 'zOffsetMax': 0.19946700285313645, 'gravityCst': 1.0, 'distribution': 'hernquist', 'offset': [-0.6236146941192648, -0.7243935992761448, 0.3226843421281147], 'initial_vel': [0.011433144869920045, 0.09702209276002413, 0.009911821539782359], 'clockwise': 1, 'angle': [6.2248188789132115, 5.619646743140335, -2.8855878478980963]}


100%|██████████| 1000/1000 [00:26<00:00, 37.33it/s]


In [11]:
def animate3d(scene):
    scene = json.loads(scene)
    mass_save = np.array(scene['masses'])
    type_save = scene['types']
    bh_num = np.sum(np.array([1 if t == 'black hole' else 0 for t in type_save]))
    pos_save = np.array([f['pos'] for f in scene['frames']])

    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.set_zlim(-2, 2)
    
    
    
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')


    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])




    # get index of 2 biggest masses
    bh_index = np.argsort(mass_save)[::-1][:bh_num]

    masses_without_bh = np.delete(mass_save, bh_index)

    max_mass = np.max(masses_without_bh)
    min_mass = np.min(masses_without_bh)

    
    # Create the colormap (inferno)
    cmap = plt.cm.autumn
    colors = np.zeros((pos_save.shape[0], 4))


    for i in range(pos_save.shape[1]):
        if i in bh_index:
            colors[i] = [0, 0, 0, 1]
        else:
            colors[i] = cmap((mass_save[i] - min_mass) / (max_mass - min_mass))
    
    # shape of pos_save is (num_frames, num_particles, 3)
    scat = ax.scatter(pos_save[0, :, 0], pos_save[0, :, 1], pos_save[0, :, 2], c=colors)

    def update(i):
        # Update the positions of the particles
        scat._offsets3d = (pos_save[i, :, 0], pos_save[i, :, 1], pos_save[i, :, 2])
        
        # Optionally update sizes if mass_save changes per timestep (if desired)
        if mass_save is not None:
            masses = mass_save[:pos_save.shape[0]]
            masses = 100 * np.sqrt(masses)
            masses[bh_index] = 10
            
            scat.set_sizes(masses)



    ani = animation.FuncAnimation(fig, update, frames=pos_save.shape[2], interval=10)
    plt.close(fig)
    return HTML(ani.to_html5_video())



In [15]:
animate3d(scene)

IndexError: index 806 is out of bounds for axis 0 with size 654